# ДЗ №4
### *виконала студентка групи ФБ-33 Журавльова Марія* ###

Нагадаю, що мій набір даних - TMDB 10000 Movies Dataset (https://www.kaggle.com/datasets/i0xc0d3x00000/tmdb-10000-movies-dataset)

**Опис колонок:**

- title - назва фільму;
- overview - опис сюжету фільму;
- release_date - оригінальна дата випуску;
- vote_average - середній рейтинг фільму;
- vote_count - кількість отриманих голосів;
- original_language - мова зйомки;
- popularity - індекс популярності.

**Прокласифікувати методом К найближчих судідів:**
- Поділити вибірку на тестову та начальну;

Для цього повторюю кроки з дз №3:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# завантажую дані
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\university\МТАД\movie_tmdb_clean.csv")
x = df[['vote_average', 'vote_count']]

# маштабування (стандартизація даних)
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

# k=3
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(scaled_x)

df_clusters = x.copy()
df_clusters['clusters'] = clusters

subset = df_clusters[df_clusters['clusters'].isin([1, 2])].copy()

# нова мітка для SVM (1->0, 2->1)
subset['label'] = subset['clusters'].map({1:0, 2:1})

X = subset[['vote_average', 'vote_count']]
Y = subset['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# розділення на набори train та test
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=100)

- Розрахувати скільки потрібно обрати найближчих записів, щоб точність була не нижче 85%;

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

k_values = range(1, 31)
accuracies = []
optimal_k = 0

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(Y_test, y_pred)
    accuracies.append(acc)
    if acc >=0.85 and optimal_k == 0:
        optimal_k = k

print(f"Мінімальна кількість К для точності >= 85%: {optimal_k}")

Мінімальна кількість К для точності >= 85%: 1


- Порахувати точність для тестової вибірки;

In [3]:
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)
knn_optimal.fit(X_train, Y_train)
y_test_pred = knn_optimal.predict(X_test)
test_accuracy = accuracy_score(Y_test, y_test_pred)

print(f"Точність: {test_accuracy}")

Точність: 1.0


- Зробити крос валідацію на 3 блоках;

In [4]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(knn_optimal, X_scaled, Y, cv=3, scoring='accuracy')
print("Результати:")
for i, score in enumerate(cv_scores, 1):
    print (f" - Блок {i:<2} : {score:.3f}")
print(f"Середня точність крос-валідації: {cv_scores.mean():.3f}")
print(f"Стандартне відхилення: {cv_scores.std():.3f}")

Результати:
 - Блок 1  : 0.734
 - Блок 2  : 1.000
 - Блок 3  : 0.799
Середня точність крос-валідації: 0.844
Стандартне відхилення: 0.113


**Висновки.** Після тестування різних значень K було знайдено, що при K=1 точність класифікації на тестовій виборці досягає 100%, що вказує на дуже чітке розділення даних між класами. Незважаючи на високу середню точність крос-валідації (~84%), спостерігається помітне стандартне відхилення (0.113) та значний розкид результатів (від 0.734 до 1). Це свідчить про те, що модель KNN з K=1 хоч і демонструє високу точність, проте все ж є недостатньо стабільною через надмірну чутливість до вибірки даних.

Для наочності проведу повторну класифікацію з K=3:

In [7]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)
y_test_pred = knn.predict(X_test)
test_accuracy = accuracy_score(Y_test, y_test_pred)

print(f"Точність: {test_accuracy}")

Точність: 1.0


In [8]:
cv_scores = cross_val_score(knn, X_scaled, Y, cv=3, scoring='accuracy')
print("Результати:")
for i, score in enumerate(cv_scores, 1):
    print (f" - Блок {i:<2} : {score:.3f}")
print(f"Середня точність крос-валідації: {cv_scores.mean():.3f}")
print(f"Стандартне відхилення: {cv_scores.std():.3f}")

Результати:
 - Блок 1  : 0.714
 - Блок 2  : 1.000
 - Блок 3  : 0.794
Середня точність крос-валідації: 0.836
Стандартне відхилення: 0.121


Бачимо, що використання більшого K призводить до зниження середньої точності та збільшення середнього відхилення. Отже, збільшення К лише погіршує узагальнюючу здатність моделі. Можна зробити висновок, що не завжди доцільно уникати малого К, особливо якщо ми впевнені, що наш набір даних є чистим (це ми з'ясували у дз №3). Таким чином, K=1 був дійсно оптимальним компромісом між точністю та ризиком нестабільності у цьому випадку.